# DESP User Workflow Data Collections

## Overview

This document serves as a quick reference guide of how to access **DESP User Workflow Processing** environment APIs, using Python, from the DESP User Workflow Development environment. This guide will show how to create a collection and upload data to it.

### Steps
1. Preparation
2. Create a Collection
3. Upload Data to a Collection

## Preparation

These preparation blocks have to be executed as a prerequisite for every subsequent step to work correctly, as these variables will be used to configure each HTTP request used in these examples.


> IMPORTANT NOTE: Please ensure to provide a valid username and token pair.

In [ ]:
import requests
import base64
import json
import tempfile
import zipfile
from pathlib import Path
from datetime import datetime
from datetime import timedelta
import getpass

In [ ]:
username = getpass.getpass()

In [ ]:
token = getpass.getpass()

In [ ]:
credentials = username + ':' + token

authorization = "Basic " + base64.b64encode(bytes(credentials, 'utf-8')).decode('utf-8')
headers = {'Authorization':authorization,'Content-Type':'application/hal+json;charset=UTF-8'}
base_url = "https://userwkfl.dev.desp.space"

## Create a Collection

For creating a new collection we need to provide a JSON object with its details along with the POST HTTP request.
Inside `coll` below you can see how the JSON object is made, and feel free to modify the name or description.

> Be mindful when running this one as if a collection with the same name is already present in the platform the command will fail with HTTP 409, so try with a different collection or delete the old one to test it a second time.

In [ ]:
coll={'name': "test-jupytr-coll", 'description': "no", 'fileType': "REFERENCE_DATA", 'productsType': "no"}

url = base_url + "/secure/api/v2.0/collections/"
run_request = requests.post(url, headers=headers, json=coll, verify=True)
run_request_dict = json.loads(run_request.text)

if run_request.status_code >= 400:
  print(str(run_request.status_code) + " ERROR: Could not create Collection")
else:
  print(str(run_request.status_code) + " Collection successfully created")

In [ ]:
run_request_dict

## Upload Data to a Collection

For this step a file (`filename`) is created form scratch and uploaded, feel free to modify it with an already existing one and if you do remove the `truncate()` line.

The object (`uploads`) has to be created with the structure below and passed under the `files` field of the http request to work correctly.

We're using the `refData` API to upload also specifying the collection and the file type.

In [ ]:
headers = {'Authorization': authorization} # Content-Type can't be in this request's headers list
file_type = 'OTHER'
collection_identifier = "eopaasbb235018ba8342dda1df91a5c3678395" # test-jupytr-file collection
filename = "uploadTestFile.tif"
metadata = {} # Optional metadata fields here

with open(filename, "w+b") as upload:
    upload.truncate(1024) # Just to produce an arbitrary 1kb file
    uploads={
        'file':(filename, upload, 'image/tif'),
        'userProperties': (None, json.dumps(metadata), 'application/json')
    }

    url = base_url + f"/secure/api/v2.0/platformFiles/refData?fileType={file_type}&collection={collection_identifier}"
    run_request = requests.post(url, headers=headers, files=uploads, verify=True)
    run_request_dict = json.loads(run_request.text)

In [ ]:
run_request_dict